# 재활용 Chatbot

# 1. 패키지 설치

In [ ]:
#%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-chroma

# 2. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./data/recycle.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
from langchain_pinecone import PineconeVectorStore

index_name='recycle-index'
chunk_size = 100  # 각 요청에 포함할 문서의 수 (예시로 100 지정)
for i in range(0, len(document_list), chunk_size):
    chunk_documents = document_list[i:i + chunk_size]
    database = PineconeVectorStore.from_documents(chunk_documents, embedding, index_name=index_name)

pinecone -> Database -> Indexes Records에서 결과 확인 가능.

In [ ]:
from pinecone import Pinecone
import os

pinecone_api_key = os.environ.get("PINECONE_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(index_name)
database = PineconeVectorStore(index=index, embedding=embedding)

# 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [ ]:
query = '우유를 마시고 난 후 재활용은 어떻게 하나요'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
retrieved_docs

[Document(id='6b1e9e0f-af5f-42c6-80c7-365c11bab9da', metadata={'source': './data/recycle.docx'}, page_content='| \\-\xa0골판지상자는 비닐코팅 부분, 상자에 붙어있는 테이프·철핀 등을 제거하고 압착 후 끈으로 묶는 등 다른 종이류와 섞이지 않게 배출 |\n\n| 합성                                                                         | 스티로폼 완충재                                                    | 내용물을 비우고 부착상표 등을 제거하고, 음식물이 묻어 있지 않도록 깨끗이 씻어서 배출          |\n\n| 수지류                                                                        |\n\n| 전지류                                                                        | 수은, 망간전지                                                    | 제품에서 분리한 후 인근의 전용수거함에 배출                                  |\n\n| 전자제품                                                                       | 소형가전                                                        | 이물질을 제거한 후 배출                                             |\n\n| 형광등                                                                        | 형광등                           

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message

{'query': '우유를 마시고 난 후 재활용은 어떻게 하나요',
 'result': '우유팩은 내용물을 깨끗이 비우고, 부착된 라벨 등을 제거한 후 압착하여 끈으로 묶어 다른 종이류와 섞이지 않게 배출합니다. 종이팩은 일반 종이와 별도로 분리하여 재활용 처리해야 합니다.'}